In [ ]:
# optimize pp and pca_number


%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import matplotlib as mpl
mpl.rc("figure", dpi=200)

In [ ]:
%time 
# load the data
from natto import input 
from natto import process
from ubergauss import tools
import numpy as np

dataset_names = input.get41names()#[:6]
def load_and_score(n,preprocessingmethod='natto'):
    data = input.load100(n,path = '/home/ubuntu/repos/natto/natto/data',remove_unlabeled=True)
    gs = process.annotate_genescores(data, mingenes = 200, selector = preprocessingmethod)
    #gs.X = np.expm1(gs.X)
    return gs 

#annotated = 
from ubergauss import tools
dists = tools.loadfile( 'dist41.dmp')
#np.fill_diagonal(dists,-999)


In [ ]:
from natto.optimize.x40 import assign 
from sklearn.metrics import adjusted_rand_score
from functools import partial
import scanpy as sc

def predictive_performance_estimate(dataset_pair):
    data_target,data_source = dataset_pair
    '''
    propagate labels
    score 
    '''
    scores = []
    for i in range(1):
        subsampled_target = sc.pp.subsample(data_target, copy=True, n_obs = 1000)
        subsampled_source = sc.pp.subsample(data_source, copy=True, n_obs = 1000)
        merged_data=process.merge([subsampled_target,subsampled_source],umaps=[], pca=20, sortfield=1)

        score = assign.diffuseandscore(merged_data.data[0].obs['true'],
                                       merged_data.data[1].obs['true'],
                                       merged_data.projections[1], 
                                       neighbors= 60)
        
        scores.append(score)
    return np.mean(scores)
    
method_scores = []
list_of_method = "natto seurat_v3 cell_ranger seurat".split()

for method in list_of_methods[:1]:
    f= partial(load_and_score, preprocessingmethod=method)
    adataL = tools.xmap(f,dataset_names)
    performance_scores = tools.xmap(predictive_performance_estimate, ((adataL[i], adataL[np.argmax(dists[i])])  for i in range(len(names)) ))
    method_scores.append(np.mean(performance_scores))

print(method_scores) 